In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [224]:
from torch.utils.data import DataLoader, Dataset, random_split
import torch
import torch.nn as nn
from torchvision.io import read_image, ImageReadMode
import torchvision.transforms as transforms
from torchvision import datasets
from torch import optim
import os
import torchvision

In [253]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [265]:
class CustomDataset(Dataset):
    def __init__(self, labels_file, imgs_dir, transform=None, target_transform=None):
        self.labels = pd.read_csv(labels_file)
        self.imgs_dir = imgs_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img_path = os.path.join(self.imgs_dir, self.labels.iloc[idx, 0])
        image = read_image(img_path).float()
        label = torch.tensor(self.labels.iloc[idx, 1], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [266]:
CustomDataset('data/Valid/Validation.csv', 'data/Valid/').__getitem__(0)

(tensor([[[255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          ...,
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.]],
 
         [[255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          ...,
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.]],
 
         [[255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          ...,
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  ..., 255., 255., 255.],
          [255., 255., 255.,  .

In [267]:
training_dataset = CustomDataset('data/Train/Train.csv', 'data/Train/')
validation_dataset = CustomDataset('data/Valid/Validation.csv', 'data/Valid/')

In [176]:
training_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [177]:
from torch import optim
import torch.nn as nn

In [9]:
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3*240*240, 2),
    nn.LogSoftmax(),
).cuda()

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [268]:
def train(dl, model, loss_fn, opt):
    size = len(dl.dataset)
    model.train()
    for batch, (image, label) in enumerate(dl):
        image = image.to(device)
        label = label.to(device)
        pred = model(image)
        loss = loss_fn(pred, label)
        loss.backward()
        opt.step()
        opt.zero_grad()

        if batch % 5==0:
            print(f"loss: {loss.item()}    [{(batch+1)*len(image)}/{size}]")

In [269]:
def evaluate(dl, model, loss_fn, opt):
    size = len(dl.dataset)
    num_batches = len(dl)
    model.eval()
    test_loss, total_correct = 0, 0
    with torch.no_grad():
        for image, label in dl:
            image, label = image.to(device), label.to(device)
            pred = model(image)
            loss = loss_fn(pred, label)
            test_loss += loss.item()
            correct = (pred.argmax(1) == label).sum().item()
            total_correct += correct        
    average_loss = test_loss / num_batches
    accuracy = total_correct / size
    print(f"Accuracy = {(100*accuracy):>0.1f}%, Avg_Loss = {average_loss}")

In [183]:
for epoch in range(5):
    print(f"====== Epoch {epoch+1} ======")
    train(training_loader, net, loss_fn, optimizer)
    evaluate(validation_loader, net, loss_fn, optimizer)
print("Done!")

====== Epoch 1 ======
loss: 2816.986328125    [32/1569]


C:\Users\Omar\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 1835.505859375    [192/1569]
loss: 4528.20849609375    [352/1569]
loss: 1941.0341796875    [512/1569]
loss: 142.69036865234375    [672/1569]
loss: 1406.876953125    [832/1569]
loss: 2649.80078125    [992/1569]
loss: 1848.65380859375    [1152/1569]
loss: 2213.591064453125    [1312/1569]
loss: 853.7764282226562    [1472/1569]
Accuracy = 73.3%, Avg_Loss = 2222.459181565505
====== Epoch 2 ======
loss: 1905.634033203125    [32/1569]
loss: 2974.205078125    [192/1569]
loss: 2101.767578125    [352/1569]
loss: 621.0003051757812    [512/1569]
loss: 353.7567443847656    [672/1569]
loss: 428.6824951171875    [832/1569]
loss: 150.85137939453125    [992/1569]
loss: 214.19943237304688    [1152/1569]
loss: 544.4990234375    [1312/1569]
loss: 176.23410034179688    [1472/1569]
Accuracy = 81.2%, Avg_Loss = 988.8099553034856
====== Epoch 3 ======
loss: 447.2850341796875    [32/1569]
loss: 460.72930908203125    [192/1569]
loss: 301.57977294921875    [352/1569]
loss: 438.7090759277344    [512/1569]
l

KeyboardInterrupt: 

# CNN

In [270]:
train_transform = transforms.Compose([ 
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(0.1),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

test_transform = transforms.Compose([
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
])

In [271]:
training_dataset = CustomDataset('data/Train/Train.csv', 'data/Train/', transform=train_transform, target_transform=test_transform)
validation_dataset = CustomDataset('data/Valid/Validation.csv', 'data/Valid/', transform=train_transform, target_transform=test_transform)

In [272]:
training_loader = DataLoader(training_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [273]:
cnn1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2),
        nn.MaxPool2d(kernel_size=2),
        
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 5, padding = 2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2),
        
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 5, padding = 2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2),
        nn.Flatten(),
        
        nn.Linear(115200, 1000),
        nn.BatchNorm1d(1000),
        
        nn.ReLU(),
        nn.Linear(1000, 1000),
        nn.BatchNorm1d(1000),
        
        nn.ReLU(),
        nn.Linear(1000, 1000),
        nn.BatchNorm1d(1000),
        
        nn.ReLU(),
        nn.Linear(1000, 1000),
        nn.BatchNorm1d(1000),
        
        nn.ReLU(),
        nn.Linear(1000, 2),
        nn.BatchNorm1d(2),
).cuda()

In [274]:
from torchsummary import summary

def print_summary(model, dim):
    summary(model, (3, dim, dim))

In [275]:
print_summary(cnn1, 240)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 240, 240]           2,432
         MaxPool2d-2         [-1, 32, 120, 120]               0
            Conv2d-3         [-1, 64, 120, 120]          51,264
              ReLU-4         [-1, 64, 120, 120]               0
         MaxPool2d-5           [-1, 64, 60, 60]               0
            Conv2d-6          [-1, 128, 60, 60]         204,928
              ReLU-7          [-1, 128, 60, 60]               0
         MaxPool2d-8          [-1, 128, 30, 30]               0
           Flatten-9               [-1, 115200]               0
           Linear-10                 [-1, 1000]     115,201,000
      BatchNorm1d-11                 [-1, 1000]           2,000
             ReLU-12                 [-1, 1000]               0
           Linear-13                 [-1, 1000]       1,001,000
      BatchNorm1d-14                 [-

In [276]:
for epoch in range(10):
    print(f"====== Epoch {epoch+1} ======")
    train(training_loader, net, loss_fn, optimizer)
    evaluate(validation_loader, net, loss_fn, optimizer)
print("Done!")

====== Epoch 1 ======


TypeError: Tensor is not a torch image.